# Travailler avec les modèles de la famille Meta

## Introduction

Cette leçon abordera :

- L’exploration des deux principaux modèles de la famille Meta - Llama 3.1 et Llama 3.2
- La compréhension des cas d’utilisation et des scénarios pour chaque modèle
- Un exemple de code pour illustrer les fonctionnalités uniques de chaque modèle

## La famille de modèles Meta

Dans cette leçon, nous allons explorer 2 modèles de la famille Meta, aussi appelée « Llama Herd » - Llama 3.1 et Llama 3.2

Ces modèles existent en plusieurs variantes et sont disponibles sur le marketplace Github Model. Voici plus d’informations sur l’utilisation des modèles Github pour [prototyper avec des modèles d’IA](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

Variantes de modèles :
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*Remarque : Llama 3 est également disponible sur Github Models mais ne sera pas abordé dans cette leçon*


## Llama 3.1

Avec 405 milliards de paramètres, Llama 3.1 appartient à la catégorie des LLM open source.

Ce modèle est une amélioration par rapport à la version précédente Llama 3 en offrant :

- Une fenêtre de contexte plus grande - 128k tokens contre 8k tokens
- Un nombre maximal de tokens en sortie plus élevé - 4096 contre 2048
- Un meilleur support multilingue - grâce à l’augmentation du nombre de tokens d’entraînement

Ces améliorations permettent à Llama 3.1 de gérer des cas d’usage plus complexes lors de la création d’applications GenAI, notamment :
- Appels de fonctions natifs - la capacité d’appeler des outils et fonctions externes en dehors du flux de travail du LLM
- Meilleures performances RAG - grâce à la fenêtre de contexte plus large
- Génération de données synthétiques - la capacité de créer des données efficaces pour des tâches comme l’ajustement fin


### Appel natif de fonctions

Llama 3.1 a été entraîné pour être plus efficace dans l’utilisation de fonctions ou d’outils. Il dispose également de deux outils intégrés que le modèle peut identifier comme nécessaires selon la demande de l’utilisateur. Ces outils sont :

- **Brave Search** – Permet d’obtenir des informations à jour, comme la météo, en effectuant une recherche sur le web
- **Wolfram Alpha** – Permet de réaliser des calculs mathématiques complexes, ce qui évite d’avoir à écrire vos propres fonctions.

Vous pouvez aussi créer vos propres outils personnalisés que le LLM pourra utiliser.

Dans l’exemple de code ci-dessous :

- Nous définissons les outils disponibles (brave_search, wolfram_alpha) dans le prompt système.
- Nous envoyons un prompt utilisateur qui demande la météo dans une certaine ville.
- Le LLM répondra par un appel à l’outil Brave Search, qui ressemblera à ceci : `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Remarque : Cet exemple ne fait qu’appeler l’outil. Si vous souhaitez obtenir les résultats, il faudra créer un compte gratuit sur la page de l’API Brave et définir la fonction vous-même.*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Bien que Llama 3.1 soit un LLM, l’une de ses limites est la multimodalité. Autrement dit, la capacité d’utiliser différents types d’entrées, comme des images, en guise d’invites et de fournir des réponses. Cette fonctionnalité fait partie des principaux atouts de Llama 3.2. Parmi les nouveautés, on retrouve également :

- Multimodalité – capable d’analyser à la fois des invites textuelles et des images
- Variantes de taille petite à moyenne (11B et 90B) – cela offre des options de déploiement flexibles,
- Variantes texte uniquement (1B et 3B) – cela permet au modèle d’être utilisé sur des appareils mobiles ou en périphérie, avec une faible latence

La prise en charge de la multimodalité marque une avancée majeure dans le domaine des modèles open source. L’exemple de code ci-dessous utilise à la fois une image et une invite textuelle pour obtenir une analyse de l’image par Llama 3.2 90B.

### Prise en charge de la multimodalité avec Llama 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## L'apprentissage ne s'arrête pas ici, poursuivez votre parcours

Après avoir terminé cette leçon, consultez notre [collection d'apprentissage sur l'IA générative](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) pour continuer à approfondir vos connaissances en IA générative !



---

**Avertissement** :  
Ce document a été traduit à l’aide du service de traduction par IA [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d’assurer l’exactitude de la traduction, veuillez noter que les traductions automatiques peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d’origine doit être considéré comme la source faisant autorité. Pour les informations critiques, il est recommandé de recourir à une traduction humaine professionnelle. Nous déclinons toute responsabilité en cas de malentendus ou d’interprétations erronées résultant de l’utilisation de cette traduction.
